# Hollender 3D image analysis
## Author: Miles Roberts
## Date: 2021-03-30
## Goals:
* Read in .ply files
* Process images to get baseline skeletons that all subgroups will work with
* Measure cool phenotypes
## Link to cookbook: http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html#

## Mess around with ply files

In [ ]:
#import pptk
import numpy as np
import plyfile
data = plyfile.PlyData.read('./Hollender_arabidopsis/arabidopsis lab test 4 (col).ply')['vertex']

In [ ]:
print(data)

In [ ]:
xyz = np.c_[data['x'], data['y'], data['z']]
rgb = np.c_[data['red'], data['green'], data['blue']]
n = np.c_[data['nx'], data['ny'], data['nz']]

In [ ]:
print(xyz)

In [ ]:
import numpy as np
import open3d as o3d

# Read .ply file
input_file = "./Hollender_arabidopsis/arabidopsis_lab_test_4_col.ply"
pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud

# Visualize the point cloud within open3d
o3d.visualization.draw_geometries([pcd]) 

# Convert open3d format to numpy array
# Here, you have the point cloud in numpy format. 
point_cloud_in_numpy = np.asarray(pcd.points) 

In [ ]:
#Downsampling
downpcd = pcd.voxel_down_sample(voxel_size=0.01)
o3d.visualization.draw_geometries([downpcd])

In [ ]:
import matplotlib as plt
#clustering
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.pyplot.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd])

In [ ]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

## Bring methods together to isolate plant

### Try downsampling before plane segmentation

In [1]:
import numpy as np
import open3d as o3d

# Read .ply file
input_file = "./Hollender_arabidopsis/arabidopsis_lab_test_4_col.ply"
pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud

# Visualize the point cloud within open3d
#o3d.visualization.draw_geometries([pcd]) 

# Convert open3d format to numpy array
# Here, you have the point cloud in numpy format. 
point_cloud_in_numpy = np.asarray(pcd.points) 

#Downsampling
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
o3d.visualization.draw_geometries([downpcd])

#Identify plane in downsampled figure
plane_model, inliers = downpcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = downpcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = downpcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

Plane equation: 0.01x + -0.04y + 1.00z + 0.55 = 0


In [2]:
#Write function to take pcd object and plane formula then subset points
def remove_plane(pcdObject, modelArray, upr, lwr):
    #Write list for output
    goodIndices = []
    #Convert pcd object to numpy array, save points; normals; and colors
    pcdPoints = np.asarray(pcdObject.points)
    pcdNormals = np.asarray(pcdObject.normals)
    pcdColors = np.asarray(pcdObject.colors)
    #Extract coefficients for plane equation
    a = plane_model[0]
    b = plane_model[1]
    c = plane_model[2]
    d = plane_model[3]
    #evaluate plane equation for each point
    for i in range(len(pcdPoints)):
        x = pcdPoints[i][0]
        y = pcdPoints[i][1]
        z = pcdPoints[i][2]
        if(a*x + b*y + c*z + d < upr and a*x + b*y + c*z + d > lwr):
            goodIndices.append(i)
    #subset array, convert array back to cloud
    pcdNew = o3d.geometry.PointCloud()
    pcdNew.points = o3d.utility.Vector3dVector(pcdPoints[goodIndices])
    pcdNew.normals = o3d.utility.Vector3dVector(pcdNormals[goodIndices])
    pcdNew.colors = o3d.utility.Vector3dVector(pcdColors[goodIndices])
    return pcdNew

In [3]:
noPlanePcd = remove_plane(downpcd, plane_model, 0.1, 0.005)
o3d.visualization.draw_geometries([noPlanePcd])

### Try not downsampling before plane segmentation

In [4]:
#Identify plane in downsampled figure
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

Plane equation: 0.01x + -0.04y + 1.00z + 0.55 = 0


In [5]:
noPlanePcd = remove_plane(pcd, plane_model, 0.1, 0.005)
o3d.visualization.draw_geometries([noPlanePcd])